In [2]:
import numpy as np
from PIL import Image, ImageOps

import os
import json
import math
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rcParams['figure.figsize'] = (20, 20)

In [3]:
DIVISOR = 32
RAW_PATH = os.path.join('data', 'raw', 'numpy')
SAVE_PATH = os.path.join('data', 'complete')

raw_files = [['inline_data.npz', 'inline_faults_thick_7.npz']]

fault_path = os.path.join(SAVE_PATH, 'fault')
seis_path = os.path.join(SAVE_PATH, 'seis')
img_cnt = 0
metadata = []
for data_names in raw_files:
    print(data_names)
    data = np.load(os.path.join(RAW_PATH, data_names[0]))['arr_0']
    min_data_value = np.min(data)
    max_data_value = np.max(data)
    data = (data - min_data_value) / (max_data_value - min_data_value) * 255

    labels = np.load(os.path.join(RAW_PATH, data_names[1]))['arr_0']
    assert data.shape == labels.shape

    for iline_num in range(np.shape(data)[0]):
        data_slice = data[iline_num]
        labels_slice = labels[iline_num]

        timelines, jlines,  = np.shape(data_slice)
        data_slice, data_padding = pad_data(data_slice, DIVISOR)
        labels_slice, labels_padding = pad_data(labels_slice, DIVISOR)
        labels_slice = labels_slice * 255

        data_img = ImageOps.grayscale(Image.fromarray(data_slice))
        label_img = ImageOps.grayscale(Image.fromarray(labels_slice))
        metadata.append({
                'data': f'{img_cnt}.png',
                'label': f'{img_cnt}.png',
                'padding': data_padding
            })
        data_img.save(os.path.join(seis_path, f'{img_cnt}.png'))
        label_img.save(os.path.join(fault_path, f'{img_cnt}.png'))
        img_cnt += 1
with open(os.path.join(SAVE_PATH, 'metadata.json'), 'w') as file:
    json.dump(metadata, file)

['inline_data.npz', 'inline_faults_thick_7.npz']


In [17]:
data = np.load(os.path.join('data', 'raw', 'numpy', 'inline_faults_thick_7.npz'))['arr_0']

In [18]:
markup = {
    'train': [],
    'test': []
}
index_shift = 312 # Первый инлайн в исходном segy имеет индекс 312
train_slices = np.linspace(10, np.shape(data)[0], 12, endpoint=False, dtype='int')
test_slices = [50, 200, 370, 520]
for stage in ['train', 'test']:
    slices = train_slices if stage == 'train' else test_slices
    for slice_num in slices:
        data_slice = data[slice_num]
        data_slice = data_slice.tolist()
        markup[stage].append({'direction': 'iline', 'slice_num': int(slice_num + index_shift), 'slice': data_slice})
with open('markup.sflt', 'w') as file:
    json.dump(markup, file)